# **Meta-Optimization for Automating Model**

## **Installing and Importing Libraries**

In [1]:
# installing packages
!pip install hyperopt
!pip install hyperas
!pip install autokeras
!pip install tensorflow 

# array processing
import numpy as np

# deep learning staple libraries
import tensorflow as tf
from tensorflow import keras

# meta-optimization
import hyperopt
import hyperas

# neural architecture search
import autokeras as ak

2022-02-23 00:28:47.298230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-23 00:28:47.298284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## **Loading Data**

For data, we'll use the CIFAR-10 dataset, with a small adaptation to decrease the size of the dataset for faster meta-optimization training - only data instances with a label of 0 or 1 are included.

In [2]:
# load cifar-10 data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# get training data
valid_train_indices = (y_train.reshape(len(y_train))==0)|(y_train.reshape(len(y_train))==1)
x_train = x_train[valid_train_indices]
y_train = y_train[valid_train_indices]

# get testing data
valid_test_indices = (y_test.reshape(len(y_test))==0)|(y_test.reshape(len(y_test))==1)
x_test = x_test[valid_test_indices]
y_test = y_test[valid_test_indices]

## **HyperOpt**
### **Finding Minimum of a Continuous Function**

Code to use Bayesian optimization via TPE algorithm to find the minimum of the function $f(x)=(x-1)^2$.

In [3]:
# define the search space
from hyperopt import hp
space = {'x':hp.normal('x', mu=0, sigma=10)}

# define objective function
def obj_func(params):
    return (params['x']-1)**2

# perform minimization procedure
from hyperopt import fmin, tpe
best = fmin(obj_func, space, algo=tpe.suggest, max_evals=500)

100%|██████████| 500/500 [00:01<00:00, 352.03trial/s, best loss: 4.500073158764411e-06]


### **Finding Minimum of a Non-continuous Function Using Statuses**

Code to find the minimum of the function $\left|\frac{1}{x}\right| + x^2$, a function that is undefined at $x=0$, to demonstrate the usage of `ok` and `fail` statuses.

In [4]:
# define the search space
from hyperopt import hp
space = {'x':hp.normal('x', mu=0, sigma=10)}

# define objective function
def obj_func(params):
    if params['x']==0:
        return {'status':'fail'}
    return {'loss':np.abs(1/params['x'])+params['x']**2,
            'status':'ok'}

# perform minimization procedure
from hyperopt import fmin, tpe
best = fmin(obj_func, space, algo=tpe.suggest, max_evals=500)

100%|██████████| 500/500 [00:01<00:00, 355.58trial/s, best loss: 1.8898829304936338]


### **Finding Optimal Optimizer and Learning Rate in CNN**

In [5]:
# define search space
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
optimizers = [Adam, RMSprop, SGD]
space = {'optimizer':hp.choice('optimizer',optimizers),
         'lr':hp.lognormal('lr', mu=0.005, sigma=0.001)}

# import necessary model and layers
from keras.models import Sequential
import keras.layers as L

# build objective function
def objective(params):

    # build model
    model = Sequential()
    model.add(L.Input((32,32,3)))
    for i in range(4):
        model.add(L.Conv2D(32, (3,3), activation='relu'))
    model.add(L.Flatten())
    model.add(L.Dense(64, activation='relu'))
    model.add(L.Dense(1, activation='sigmoid'))

    # compile
    optimizer = params['optimizer'](lr=params['lr'])
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # fit
    model.fit(x_train, y_train, epochs=1, verbose=0) # increase epochs for better performance

    # evaluate accuracy (second elem. w/ .evaluate())
    acc = model.evaluate(x_test, y_test, verbose=0)[1]

    # return negative of acc such that smaller = better
    return -acc

# perform search
best = fmin(objective, space, algo=tpe.suggest, max_evals=1) # increase evals for better performance

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

2022-02-23 00:36:22.960318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-23 00:36:22.960399: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-23 00:36:22.960462: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-61e195dae136b0d2eea966b8): /proc/driver/nvidia/version does not exist
2022-02-23 00:36:22.962621: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.8/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The

100%|██████████| 1/1 [00:16<00:00, 16.44s/trial, best loss: -0.5]


### **Finding Optimal Model Architecture**

In [6]:
# specify important parameters for search space
min_num_convs = 3
max_num_convs = 8
min_num_dense = 2
max_num_dense = 5

# obtain list of dropout rates
conv_drs, dense_drs = [], []
for layer in range(max_num_convs):
    conv_drs.append(hp.normal(f'c{layer}', 0.15, 0.1))
for layer in range(max_num_dense):
    dense_drs.append(hp.normal(f'd{layer}', 0.2, 0.1))

# define search space
space = {'#convs':hp.quniform('#convs',
                              min_num_convs,
                              max_num_convs,
                              q=1),
         '#dense':hp.quniform('#dense',
                              min_num_dense,
                              max_num_dense,
                              q=1),
         'conv_dr':conv_drs,
         'dense_dr':dense_drs}

# define objective function
def objective(params):
    
    # convert set of params to list for mutability
    conv_drs = list(params['conv_dr'])
    dense_drs = list(params['dense_dr'])

    # make sure dropout rate is 0 <= r < 1
    for ind in range(len(conv_drs)):
        if conv_drs[ind] > 0.9:
            conv_drs[ind] = 0.9
        if conv_drs[ind] < 0:
            conv_drs[ind] = 0
    for ind in range(len(dense_drs)):
        if dense_drs[ind] > 0.9:
            dense_drs[ind] = 0.9
        if dense_drs[ind] < 0:
            dense_drs[ind] = 0
    
    # build model template + input
    model = Sequential()
    model.add(L.Input((32,32,3))) 

    # build convolutional component
    for ind in range(int(params['#convs'])):

        # add convolutional layer
        model.add(L.Conv2D(32, (3,3), activation='relu'))

        # add corresponding dropout rate
        model.add(L.Dropout(conv_drs[ind]))

    # add flattening for dense component
    model.add(L.Flatten()) 
    
    # build dense component
    for ind in range(int(params['#dense'])):

        # add dense layer
        model.add(L.Dense(32, activation='relu'))

        # add corresponding dropout rate
        model.add(L.Dropout(dense_drs[ind]))
        
    # add output
    model.add(L.Dense(1, activation='sigmoid'))
    
    # compile
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # fit
    model.fit(x_train, y_train, epochs=1, verbose=0) # increase epochs for better performance

    # evaluate accuracy (second elem. w/ .evaluate())
    acc = model.evaluate(x_test, y_test, verbose=0)[1]

    # return negative of acc such that smaller = better
    return -acc

# perform search
best = fmin(objective, space, algo=tpe.suggest, max_evals=1) # increase evals for better performance

100%|██████████| 1/1 [00:29<00:00, 29.29s/trial, best loss: -0.5]


## **Auto-Keras**

Note that Auto-Keras is quite memory consuming. If you run multiple meta-optimization campaigns in one session, expect for memory problems.

### **Simple Image Block**

In [7]:
# define architecture
inp = ak.ImageInput()
imageblock = ak.ImageBlock()(inp)
output = ak.ClassificationHead()(imageblock)

# aggregate into model
search = ak.AutoModel(
    inputs=inp, outputs=output, max_trials=1 # increase max trials for better performance
)

# fit
search.fit(x_train, y_train, epochs=1) # increase epochs for better performance

# export model
best_model = search.export_model()

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
INFO:tensorflow:Oracle triggered exit
313/313 [==============================] - 510s 2s/step - loss: 0.3806 - accuracy: 0.8641


2022-02-23 00:50:02.869054: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


### **Custom Search Space**

In [8]:
# define architecture
inp = ak.ImageInput()
aug = ak.ImageAugmentation(translation_factor=0.1,
                           vertical_flip=False,
                           horizontal_flip=True)(inp)
resnetblock = ak.ResNetBlock(pretrained=True,
                             version=None)(aug)
denseblock = ak.DenseBlock()(resnetblock)
output = ak.ClassificationHead()(denseblock)

# aggregate into model
search = ak.AutoModel(
    inputs=inp, outputs=output, max_trials=1 # increase max trials for better performance
)

# fit
search.fit(x_train, y_train, epochs=1) # increase epochs for better performance

# export model
best_model = search.export_model()

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
INFO:tensorflow:Oracle triggered exit
313/313 [==============================] - 27s 78ms/step - loss: 0.2997 - accuracy: 0.8765
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


### **Nonlinear Topology**

In [9]:
# define architecture
inp = ak.ImageInput()
resnetblock = ak.ResNetBlock(pretrained=True)(inp)
xceptionblock = ak.XceptionBlock(pretrained=True)(inp)
merge = ak.Merge()([resnetblock, xceptionblock])
denseblock = ak.DenseBlock()(merge)
output = ak.ClassificationHead()(denseblock)

# aggregate into model
search = ak.AutoModel(
    inputs=inp, outputs=output, max_trials=1 # increase max trials for better performance
)

# fit
search.fit(x_train, y_train, epochs=1) # increase epochs for better performance

# export model
best_model = search.export_model()

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
INFO:tensorflow:Oracle triggered exit
313/313 [==============================] - 84s 254ms/step - loss: 1.0949 - accuracy: 0.8086
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
